In [1]:
import pandas as pd

In [38]:
clima = pd.read_parquet('../data/datos_climaticos.parquet')
dengue = pd.read_csv('../data/dengue-final.csv')

In [46]:
clima.sample()

,fecha,id_estacion,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,tesion_vapor_media,radiacion_global,heliofania_efectiva,heliofania_relativa,anio,semana
373417,2003-04-26,NH0442,0.0,9.1,15.1,12.1,83.0,9.3,11.7,20.578638,8.690897,60.950577,2003,17


In [48]:
clima.fecha = pd.to_datetime(clima.fecha)
clima['anio'] = clima.fecha.dt.year
clima['semana'] = clima.fecha.dt.isocalendar().week

In [61]:
clima_groupby = clima.groupby(['id_estacion', 'anio', 'semana']).mean().reset_index()

In [50]:
clima_groupby.sample()

,id_estacion,anio,semana,fecha,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,tesion_vapor_media,radiacion_global,heliofania_efectiva,heliofania_relativa
43180,NH0416,2021,46,2021-11-18,0.106834,15.085714,28.957143,22.021428,62.571429,16.054093,18.752406,22.757068,8.657143,64.145714


In [62]:
dengue.sample()

,id_uta,departamento_nombre,provincia_nombre,ano,semanas_epidemiologicas,cantidad_casos,poblacion,estacion_id_interno
57941,82084,rosario,santa fe,2024,3.0,21.0,1316568,NH0046


In [63]:
df = pd.merge(
    dengue,
    clima_groupby,
    left_on=['estacion_id_interno', 'ano', 'semanas_epidemiologicas'],
    right_on=['id_estacion', 'anio','semana'],
    how='inner'
)

df.sample(5)

,id_uta,departamento_nombre,provincia_nombre,ano,semanas_epidemiologicas,cantidad_casos,poblacion,estacion_id_interno,id_estacion,anio,...,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,tesion_vapor_media,radiacion_global,heliofania_efectiva,heliofania_relativa
17779,86035,banda,santiago del estero,2024,11.0,108.0,171438,NH0550,NH0550,2024,...,0.026685,25.035714,36.457143,30.875000,69.785714,23.745723,30.092765,14.995377,5.522857,45.068268
252,86049,santiago del estero capital,santiago del estero,2018,21.0,1.0,292124,NH0550,NH0550,2018,...,0.000000,4.058333,23.583333,14.533333,73.250000,9.698216,12.482361,12.476101,7.616667,72.221667
7517,54119,25 de mayo,misiones,2023,20.0,1.0,29131,NH0423,NH0423,2023,...,0.000000,13.628571,25.471429,19.550000,74.714286,14.660271,16.776121,12.347766,7.085714,66.334286
2516,22133,quitilipi,chaco,2020,17.0,3.0,38714,NH0415,NH0415,2020,...,0.092546,16.985714,31.928571,24.457143,64.000000,16.794356,19.257067,16.544814,8.814286,78.712857
18079,6427,la matanza,buenos aires,2025,16.0,2.0,2509547,A872810,A872810,2025,...,0.127205,11.400000,23.614286,16.201487,84.785714,12.679190,15.759758,8.725555,3.534173,31.857143


In [74]:
columnas_drop = ['ano','semanas_epidemiologicas', 'estacion_id_interno', 'fecha']
sort_order = ['id_uta', 'anio', 'semana']

orden_columnas = ['id_uta', 'departamento_nombre', 'provincia_nombre',
 'id_estacion','anio', 'semana',
 'cantidad_casos', 'poblacion',
 'temperatura_minima', 'temperatura_maxima', 'temperatura_media',
 'precipitacion_pluviometrica',
 'humedad_media', 'rocio_medio', 'tesion_vapor_media',
 'radiacion_global', 'heliofania_efectiva', 'heliofania_relativa']

df = df.drop(columns=columnas_drop).sort_values(sort_order).drop_duplicates()[orden_columnas].reset_index(drop=True)

In [76]:
df.sample(10)

,id_uta,departamento_nombre,provincia_nombre,id_estacion,anio,semana,cantidad_casos,poblacion,temperatura_minima,temperatura_maxima,temperatura_media,precipitacion_pluviometrica,humedad_media,rocio_medio,tesion_vapor_media,radiacion_global,heliofania_efectiva,heliofania_relativa
6786,22039,2 de abril,chaco,A872958,2018,19,11.0,7844,15.814286,26.128571,20.321131,0.383353,71.857143,15.272877,17.834709,8.416809,4.952770,45.785714
7383,22133,quitilipi,chaco,NH0415,2023,44,1.0,39773,13.500000,26.228571,19.864286,0.029608,73.000000,15.452770,17.991993,17.070286,7.300000,55.700197
7840,30070,la paz,entre rios,A872914,2023,12,6.0,72990,22.142857,32.200000,26.273411,0.128221,75.000000,22.200170,24.978416,6.530409,1.319011,11.071429
10023,82049,general obligado,santa fe,A872963,2023,19,8.0,205315,6.914286,26.971429,16.017583,0.000000,64.428571,11.742492,9.730602,11.754326,6.376773,59.285714
4667,6658,quilmes,buenos aires,A872810,2023,16,10.0,686466,10.471429,22.514286,15.228273,0.077973,77.714286,11.421225,13.654619,7.838515,2.933319,26.428571
11661,86049,santiago del estero capital,santiago del estero,NH0550,2023,13,60.0,306825,19.142857,28.878571,23.032143,0.054513,84.467532,17.779459,24.310926,10.974943,3.514286,29.657143
3367,6515,malvinas argentinas,buenos aires,A872801,2023,16,3.0,369648,9.485714,22.042857,14.982243,0.096110,73.142857,10.318284,12.725034,10.541520,5.159874,46.857143
10569,82077,9 de julio,santa fe,A872962,2023,11,5.0,34502,22.228571,36.342857,28.521629,0.045378,61.285714,20.918759,24.844464,15.200640,5.765029,47.142857
2593,6434,lanus,buenos aires,A872810,2024,4,2.0,462564,20.228571,32.357143,26.228867,0.000000,59.285714,18.609781,21.628716,26.418709,11.059194,79.285714
1996,6412,jose c paz,buenos aires,A872801,2024,14,106.0,321710,12.400000,24.528571,18.165080,0.157445,78.285714,14.098491,16.200014,11.754026,6.399810,55.642857


In [77]:
df_sorted = df.sort_values(by=['id_uta', 'anio', 'semana']).copy()

climatic_cols = [
    'temperatura_minima', 'temperatura_maxima', 'temperatura_media',
    'precipitacion_pluviometrica', 'humedad_media', 'rocio_medio',
    'tesion_vapor_media', 'radiacion_global', 'heliofania_efectiva',
    'heliofania_relativa'
]

### 3. Aplicar las columnas de Lag (1, 2 y 3 semanas)
for col in climatic_cols:
    for lag in range(1, 4):  # Calcula lag1, lag2 y lag3
        new_col_name = f'{col}_lag{lag}'
        
        # Uso de groupby() y shift() para calcular el lag dentro de cada 'id_uta'
        df_sorted[new_col_name] = df_sorted.groupby('id_uta')[col].shift(lag)

In [80]:
df_sorted.dropna(inplace=True)

In [82]:
df_sorted.reset_index(drop=True)

,id_uta,departamento_nombre,provincia_nombre,id_estacion,anio,semana,cantidad_casos,poblacion,temperatura_minima,temperatura_maxima,...,tesion_vapor_media_lag3,radiacion_global_lag1,radiacion_global_lag2,radiacion_global_lag3,heliofania_efectiva_lag1,heliofania_efectiva_lag2,heliofania_efectiva_lag3,heliofania_relativa_lag1,heliofania_relativa_lag2,heliofania_relativa_lag3
0,2008,comuna 8,ciudad de buenos aires,A872810,2020,7,1.0,228953,17.695502,30.134328,...,23.412006,17.392447,17.392447,19.425206,6.938757,6.938757,7.937013,51.142857,51.142857,56.928571
1,2008,comuna 8,ciudad de buenos aires,A872810,2020,8,2.0,228953,14.612961,26.347904,...,24.017211,19.981907,17.392447,17.392447,7.841595,6.938757,6.938757,58.714286,51.142857,51.142857
2,2008,comuna 8,ciudad de buenos aires,A872810,2020,8,3.0,228953,14.612961,26.347904,...,24.017211,17.812327,19.981907,17.392447,6.905201,7.841595,6.938757,52.624060,58.714286,51.142857
3,2008,comuna 8,ciudad de buenos aires,A872810,2020,8,1.0,228953,14.612961,26.347904,...,18.818262,17.812327,17.812327,19.981907,6.905201,6.905201,7.841595,52.624060,52.624060,58.714286
4,2008,comuna 8,ciudad de buenos aires,A872810,2020,8,4.0,228953,14.612961,26.347904,...,15.009032,17.812327,17.812327,17.812327,6.905201,6.905201,6.905201,52.624060,52.624060,52.624060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12514,90091,simoca,tucuman,NH0444,2025,17,1.0,35572,15.957143,24.000000,...,27.700000,10.935289,14.736691,14.736691,3.366667,5.800000,5.800000,28.760000,48.500000,48.500000
12515,90091,simoca,tucuman,NH0444,2025,19,1.0,35572,17.342857,26.357143,...,27.700000,6.688571,10.935289,14.736691,1.428571,3.366667,5.800000,12.857143,28.760000,48.500000
12516,90091,simoca,tucuman,NH0444,2025,20,1.0,35572,15.471429,25.600000,...,14.916667,9.735714,6.688571,10.935289,3.614286,1.428571,3.366667,33.571429,12.857143,28.760000
12517,90098,tafi del valle,tucuman,NH0444,2023,12,1.0,18114,23.250000,36.700000,...,26.300000,14.736691,15.951159,15.951159,5.800000,6.550000,6.550000,48.500000,51.000000,51.000000


In [84]:
df_final = df_sorted.drop(columns=climatic_cols).reset_index(drop=True)
df_final.head()

,id_uta,departamento_nombre,provincia_nombre,id_estacion,anio,semana,cantidad_casos,poblacion,temperatura_minima_lag1,temperatura_minima_lag2,...,tesion_vapor_media_lag3,radiacion_global_lag1,radiacion_global_lag2,radiacion_global_lag3,heliofania_efectiva_lag1,heliofania_efectiva_lag2,heliofania_efectiva_lag3,heliofania_relativa_lag1,heliofania_relativa_lag2,heliofania_relativa_lag3
0,2008,comuna 8,ciudad de buenos aires,A872810,2020,7,1.0,228953,21.042857,21.042857,...,23.412006,17.392447,17.392447,19.425206,6.938757,6.938757,7.937013,51.142857,51.142857,56.928571
1,2008,comuna 8,ciudad de buenos aires,A872810,2020,8,2.0,228953,17.695502,21.042857,...,24.017211,19.981907,17.392447,17.392447,7.841595,6.938757,6.938757,58.714286,51.142857,51.142857
2,2008,comuna 8,ciudad de buenos aires,A872810,2020,8,3.0,228953,14.612961,17.695502,...,24.017211,17.812327,19.981907,17.392447,6.905201,7.841595,6.938757,52.624060,58.714286,51.142857
3,2008,comuna 8,ciudad de buenos aires,A872810,2020,8,1.0,228953,14.612961,14.612961,...,18.818262,17.812327,17.812327,19.981907,6.905201,6.905201,7.841595,52.624060,52.624060,58.714286
4,2008,comuna 8,ciudad de buenos aires,A872810,2020,8,4.0,228953,14.612961,14.612961,...,15.009032,17.812327,17.812327,17.812327,6.905201,6.905201,6.905201,52.624060,52.624060,52.624060


In [85]:
df_final.to_csv('../data/data-modelo.csv', index=False)